# Healthcare Workforce and Wait Time Analysis

## Project Overview
This analysis explores the relationship between healthcare workforce distribution across Canadian provinces and wait times for priority medical procedures. The goal is to identify patterns and potentially predict wait times based on workforce capacity.

## Data Sources
1. **Health Workforce Data (2019-2023)**: Healthcare professional counts and rates by province
2. **Wait Times Data (2025)**: Wait time information for priority medical procedures by province

In [8]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import warnings

# Configure display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8')
warnings.filterwarnings('ignore')

print("Dependencies imported successfully!")


Dependencies imported successfully!


In [10]:
# Load Wait Times Data + Clean Invalid Columns
print("=== WAIT TIMES DATA ===\n")

# Load wait times data file path
waittime_file = 'data/wait-times-priority-procedures-in-canada-2025-data-tables-en.xlsx'

def filter_relevant_waittime_data(df):
    """Clean invalid columns from wait times data"""
    original_shape = df.shape
    
    # Identify invalid columns
    invalid_columns = []
    for col in df.columns:
        col_str = str(col)
        if (col_str.startswith('Unnamed:') or 
            col_str.startswith('Column') or 
            col_str in ['nan', 'NaN'] or
            col_str.strip() == '' or
            'Unnamed' in col_str):
            invalid_columns.append(col)
    
    # Remove invalid columns
    df_clean = df.drop(columns=invalid_columns) if invalid_columns else df.copy()

    df_clean.columns = (
        df_clean.columns
        .str.strip()
        .str.lower()
        .str.replace(' ', '_')
        .str.replace(r'[^a-z0-9_]', '', regex=True)
    )

    df_clean["indicator_result"] = df_clean["indicator_result"].fillna("N/A")

    df_clean = df_clean.drop(columns=['region'])

    # Keep only years 2019–2023
    df_clean = df_clean[
        df_clean['data_year'].isin(["2019FY", "2020FY", "2021FY", "2022FY", "2023FY"])
    ]

    # Filter for reporting level = "Provincial" or "Regional"
    df_clean = df_clean[
        df_clean['reporting_level'].str.strip().str.lower().isin(['provincial'])
    ]

    if invalid_columns:
        print(f"  Wait Times Table 1: {original_shape} → {df_clean.shape} (removed {len(invalid_columns)} invalid columns)")
    else:
        print(f"  Wait Times Table 1: {original_shape} (no invalid columns)")
    
    return df_clean

# Load raw data
waittime_table1_raw = pd.read_excel(waittime_file, sheet_name='Table 1', skiprows=1, header=0)

# Clean invalid columns immediately
waittime_table1 = filter_relevant_waittime_data(waittime_table1_raw)

# Export as CSV
output_path = 'data/wait_times_priority_procedures_in_canada_2019_to_2023_fiscal_year.csv'
waittime_table1.to_csv(output_path, index=False)
print(f" Exported cleaned data to: {output_path}")

print("=" * 60)
print("TABLE 1")
print("=" * 60)
print(f"Shape: {waittime_table1.shape}")
print(f"Columns ({len(waittime_table1.columns)}): {list(waittime_table1.columns)}")

print("\nFirst 30 rows:")
display(waittime_table1.head(10))

print(f"\nData types:")
print(waittime_table1.dtypes)
print(f"\nNull values count:")
print(waittime_table1.isnull().sum())

=== WAIT TIMES DATA ===

  Wait Times Table 1: (18965, 12) → (2040, 7) (removed 4 invalid columns)
 Exported cleaned data to: data/wait_times_priority_procedures_in_canada_2019_to_2023_fiscal_year.csv
TABLE 1
Shape: (2040, 7)
Columns (7): ['reporting_level', 'province', 'indicator', 'metric', 'data_year', 'unit_of_measurement', 'indicator_result']

First 30 rows:


,reporting_level,province,indicator,metric,data_year,unit_of_measurement,indicator_result
11404,Provincial,Alberta,Bladder Cancer Surgery,50th Percentile,2019FY,Days,29.0
11405,Provincial,Alberta,Bladder Cancer Surgery,90th Percentile,2019FY,Days,68.0
11406,Provincial,Alberta,Bladder Cancer Surgery,Volume,2019FY,Number of cases,2312.0
11407,Provincial,Alberta,Breast Cancer Surgery,50th Percentile,2019FY,Days,19.0
11408,Provincial,Alberta,Breast Cancer Surgery,90th Percentile,2019FY,Days,36.0
11409,Provincial,Alberta,Breast Cancer Surgery,Volume,2019FY,Number of cases,3787.0
11410,Provincial,Alberta,CABG,50th Percentile,2019FY,Days,7.28
11411,Provincial,Alberta,CABG,90th Percentile,2019FY,Days,76.23
11412,Provincial,Alberta,CABG,Volume,2019FY,Number of cases,1100.0
11413,Provincial,Alberta,Cataract Surgery,% Meeting Benchmark,2019FY,Proportion,45.1



Data types:
reporting_level        object
province               object
indicator              object
metric                 object
data_year              object
unit_of_measurement    object
indicator_result       object
dtype: object

Null values count:
reporting_level        0
province               0
indicator              0
metric                 0
data_year              0
unit_of_measurement    0
indicator_result       0
dtype: int64


In [11]:
def convert_hours_to_days(df):
    """Convert indicator results from hours to days"""
    df_converted = df.copy()
    # Pick up both 'Hour' & 'Hours'
    mask = df_converted['unit_of_measurement'].str.lower().isin(['hour', 'hours'])
    
    # Divide by 24, round, and overwrite
    df_converted.loc[mask, 'indicator_result'] = (
        pd.to_numeric(df_converted.loc[mask, 'indicator_result'], errors='coerce')
        / 24
    ).round(2)
    
    # Update the unit
    df_converted.loc[mask, 'unit_of_measurement'] = 'Days'
    return df_converted

def clean_data(df):
    """Clean data by rounding indicator results to one decimal place and removing FY from years"""
    df_cleaned = df.copy()
    
    # Round indicator_result to one decimal place where numeric
    mask_numeric = pd.to_numeric(df_cleaned['indicator_result'], errors='coerce').notna()
    df_cleaned.loc[mask_numeric, 'indicator_result'] = (
        pd.to_numeric(df_cleaned.loc[mask_numeric, 'indicator_result'], errors='coerce')
        .round(1)
    )
    
    # Remove 'FY' from data_year
    df_cleaned['data_year'] = df_cleaned['data_year'].str.replace('FY', '')
    
    return df_cleaned

# Assuming waittime_table1 is your input DataFrame
# Apply the cleaning and conversion
waittime_table2 = clean_data(waittime_table1)
waittime_table2 = convert_hours_to_days(waittime_table2)

# Export as CSV
output_path = 'data/wait_times_priority_procedures_in_canada_hours_to_days.csv'
waittime_table2.to_csv(output_path, index=False)
print(f"Exported cleaned data to: {output_path}")

print("\nFirst 30 rows:")
display(waittime_table2.head(30))

Exported cleaned data to: data/wait_times_priority_procedures_in_canada_hours_to_days.csv

First 30 rows:


,reporting_level,province,indicator,metric,data_year,unit_of_measurement,indicator_result
11404,Provincial,Alberta,Bladder Cancer Surgery,50th Percentile,2019,Days,29.0
11405,Provincial,Alberta,Bladder Cancer Surgery,90th Percentile,2019,Days,68.0
11406,Provincial,Alberta,Bladder Cancer Surgery,Volume,2019,Number of cases,2312.0
11407,Provincial,Alberta,Breast Cancer Surgery,50th Percentile,2019,Days,19.0
11408,Provincial,Alberta,Breast Cancer Surgery,90th Percentile,2019,Days,36.0
11409,Provincial,Alberta,Breast Cancer Surgery,Volume,2019,Number of cases,3787.0
11410,Provincial,Alberta,CABG,50th Percentile,2019,Days,7.3
11411,Provincial,Alberta,CABG,90th Percentile,2019,Days,76.2
11412,Provincial,Alberta,CABG,Volume,2019,Number of cases,1100.0
11413,Provincial,Alberta,Cataract Surgery,% Meeting Benchmark,2019,Proportion,45.1


In [9]:
# Load All Workforce Data + Clean Invalid Columns (with Error Handling)
print("Loading and cleaning workforce data...")

# Check if file exists
workforce_file = 'data/health-workforce-canada-2019-2023-overview-data-tables-en.xlsx'

def clean_invalid_columns(df, table_name):
    """Clean invalid columns during loading"""
    try:
        original_shape = df.shape
        
        # Identify invalid columns
        invalid_columns = []
        for col in df.columns:
            col_str = str(col)
            if (col_str.startswith('Unnamed:') or 
                col_str.startswith('Column') or 
                col_str in ['nan', 'NaN'] or
                col_str.strip() == '' or
                'Unnamed' in col_str):
                invalid_columns.append(col)
        
        # Remove invalid columns
        df_clean = df.drop(columns=invalid_columns) if invalid_columns else df.copy()
        
        if invalid_columns:
            print(f"  {table_name}: {original_shape} → {df_clean.shape} (removed {len(invalid_columns)} invalid columns)")
        else:
            print(f"  {table_name}: {original_shape} (no invalid columns)")
        
        return df_clean
    except Exception as e:
        print(f"  ERROR cleaning {table_name}: {str(e)}")
        return df

# Define tables to load (exclude territory-only tables 12, 13, 14)
excluded_tables = [12, 13, 14]
included_tables = [i for i in range(1, 18) if i not in excluded_tables]

# Load all tables with error handling
workforce_data = {}
successful_loads = 0
failed_loads = 0

for table_num in included_tables:
    table_name = f'Table {table_num}'
    
    try:
        # Load raw data
        df_raw = pd.read_excel(workforce_file, sheet_name=table_name, skiprows=1, header=0)
        
        # Clean invalid columns immediately
        df_clean = clean_invalid_columns(df_raw, table_name)
        
        # Fix Year column to be integers instead of decimals (2019.0 → 2019)
        if 'Year' in df_clean.columns:
            df_clean['Year'] = df_clean['Year'].astype('Int64')  # Use nullable integer
            print(f"    Fixed Year column: converted to integers")
        
        # Store cleaned version in dictionary
        workforce_data[table_name] = df_clean
        
        # Create global variable with cleaned data
        var_name = f"workforce_table{table_num}"
        globals()[var_name] = df_clean
        
        successful_loads += 1
        
    except Exception as e:
        print(f"  ERROR loading {table_name}: {str(e)}")
        failed_loads += 1
        continue

def clean_numeric_artifacts(df):
    """Remove footnote symbols like *, † from numeric fields"""
    try:
        df_cleaned = df.copy()
        
        # Define common footnote symbols to remove
        footnote_symbols = ['*', '†']
        
        for col in df_cleaned.select_dtypes(include='object').columns:
            try:
                # Skip obviously non-numeric columns
                if col.lower() in ['type of professional', 'profession', 'category', 'indicator']:
                    continue
                
                # Clean the column
                cleaned_series = df_cleaned[col].astype(str)
                
                # Remove each footnote symbol explicitly
                for symbol in footnote_symbols:
                    cleaned_series = cleaned_series.str.replace(symbol, '', regex=False)
                
                # Also remove any remaining non-numeric characters except digits, dots, and minus
                cleaned_series = cleaned_series.str.replace(r'[^\d\.\-\s]', '', regex=True)
                
                # Clean up whitespace and empty strings
                cleaned_series = cleaned_series.str.strip().replace('', pd.NA)
                
                # Try to convert to numeric
                numeric_series = pd.to_numeric(cleaned_series, errors='coerce')
                
                # Only replace if we have some valid numeric values
                if not numeric_series.isna().all():
                    # Check if original had numeric-looking content
                    original_str = df_cleaned[col].astype(str)
                    has_digits = original_str.str.contains(r'\d', na=False).any()
                    
                    if has_digits:
                        df_cleaned[col] = numeric_series
                        print(f"    Cleaned symbols from column: {col}")
                
            except Exception as e:
                print(f"    WARNING: Could not clean column {col}: {str(e)}")
                continue
        
        return df_cleaned
    except Exception as e:
        print(f"  WARNING: Error cleaning numeric artifacts: {str(e)}")
        return df


# Define function for forward-filling profession names
def forward_fill_professions(df, table_name):
    """Forward-fill profession names in provincial tables (Tables 2-11)"""
    try:
        df_filled = df.copy()
        
        # Find the profession column (various possible names)
        profession_columns = ['Type of professional', 'Type of Professional', 'Professional Type', 
                            'Profession', 'profession', 'type_of_professional']
        
        profession_col = None
        for col in profession_columns:
            if col in df_filled.columns:
                profession_col = col
                break
        
        if profession_col is None:
            # Check for columns containing "professional" or "profession"
            for col in df_filled.columns:
                if 'professional' in col.lower() or 'profession' in col.lower():
                    profession_col = col
                    break
        
        if profession_col is not None:
            # Count empty/null values before forward fill
            empty_before = df_filled[profession_col].isna().sum() + (df_filled[profession_col] == '').sum()
            
            # Forward fill profession names
            df_filled[profession_col] = df_filled[profession_col].replace('', pd.NA).ffill()
            
            # Count empty/null values after forward fill
            empty_after = df_filled[profession_col].isna().sum() + (df_filled[profession_col] == '').sum()
            
            if empty_before > empty_after:
                print(f"    {table_name}: Forward-filled {empty_before - empty_after} profession names in '{profession_col}'")
            else:
                print(f"    {table_name}: No profession names needed forward-filling")
                
        else:
            print(f"    {table_name}: No profession column found - skipping forward fill")
        
        return df_filled
        
    except Exception as e:
        print(f"    ERROR: Forward-fill failed for {table_name}: {str(e)}")
        return df

# Clean numeric artifacts for all loaded workforce tables
if workforce_data:
    print(f"\nCleaning numeric artifacts...")
    for table_name, df in workforce_data.items():
        # Clean the data
        cleaned_df = clean_numeric_artifacts(df)
        
        # Update the dictionary
        workforce_data[table_name] = cleaned_df
        
        # IMPORTANT: Update the global variables with cleaned data
        table_num = int(table_name.split(' ')[1])  # Extract number from "Table X"
        var_name = f"workforce_table{table_num}"
        globals()[var_name] = cleaned_df
        print(f"    Updated global variable: {var_name}")



# Apply forward-fill to provincial tables (Tables 2-11)
if workforce_data:
    print(f"\nForward-filling profession names for provincial tables...")
    
    provincial_tables = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11]  # Tables 2-11 are provincial
    
    for table_name, df in workforce_data.items():
        table_num = int(table_name.split(' ')[1])  # Extract number from "Table X"
        
        if table_num in provincial_tables:
            # Apply forward-fill
            filled_df = forward_fill_professions(df, table_name)
            
            # Update the dictionary
            workforce_data[table_name] = filled_df
            
            # Update the global variable
            var_name = f"workforce_table{table_num}"
            globals()[var_name] = filled_df
            print(f"      Updated global variable: {var_name}")
        else:
            print(f"    {table_name}: Skipped (not a provincial table)")
    
    # Verification: Show before/after example for Table 2 (first provincial table)
    if 'Table 2' in workforce_data:
        print(f"\n🔍 VERIFICATION EXAMPLE (Table 2 - Newfoundland and Labrador):")
        table2_df = workforce_data['Table 2']
        
        # Find profession column
        profession_col = None
        for col in table2_df.columns:
            if 'professional' in col.lower() or 'profession' in col.lower():
                profession_col = col
                break
        
        if profession_col:
            print(f"Professional column: '{profession_col}'")
            print(f"Sample of forward-filled data:")
            sample_data = table2_df[[profession_col, 'Year']].head(10) if 'Year' in table2_df.columns else table2_df[profession_col].head(10)
            print(sample_data.to_string(index=False))
            
            # Count unique professions
            unique_profs = table2_df[profession_col].dropna().nunique()
            print(f"✅ Unique professions in table: {unique_profs}")
        else:
            print("Could not find profession column for verification")

# Summary
print(f"\n" + "="*60)
print("LOADING SUMMARY:")
print(f"SUCCESS: {successful_loads} tables loaded successfully")
print(f"FAILED: {failed_loads} tables failed to load")
print(f"Tables loaded: {list(workforce_data.keys()) if workforce_data else 'None'}")

if successful_loads > 0:
    print(f"\nGlobal variables created:")
    for table_num in included_tables:
        var_name = f"workforce_table{table_num}"
        if var_name in globals():
            print(f"  - {var_name}")
    
    # Verify Year column fix
    print(f"\nYear column verification:")
    for table_name, df in workforce_data.items():
        if 'Year' in df.columns:
            sample_years = df['Year'].dropna().unique()[:3]  # Show first 3 unique years
            print(f"  {table_name}: Years = {list(sample_years)} (type: {df['Year'].dtype})")
            break  # Just show one example
    
    print(f"\nAll tables are pre-cleaned and ready for analysis!")
else:
    print(f"\nERROR: No tables were loaded successfully. Please check:")
    print(f"  1. File exists: {workforce_file}")
    print(f"  2. File is not corrupted")
    print(f"  3. Required sheet names exist")


Loading and cleaning workforce data...
  Table 1: (51, 29) (no invalid columns)
  Table 2: (195, 8) (no invalid columns)
    Fixed Year column: converted to integers
  Table 3: (196, 8) (no invalid columns)
    Fixed Year column: converted to integers
  Table 4: (196, 16382) → (196, 8) (removed 16374 invalid columns)
    Fixed Year column: converted to integers
  Table 5: (196, 8) (no invalid columns)
    Fixed Year column: converted to integers
  Table 6: (197, 8) (no invalid columns)
    Fixed Year column: converted to integers
  Table 7: (195, 8) (no invalid columns)
    Fixed Year column: converted to integers
  Table 8: (195, 8) (no invalid columns)
    Fixed Year column: converted to integers
  Table 9: (195, 8) (no invalid columns)
    Fixed Year column: converted to integers
  Table 10: (196, 8) (no invalid columns)
    Fixed Year column: converted to integers
  Table 11: (196, 8) (no invalid columns)
    Fixed Year column: converted to integers
  Table 15: (16, 16384) → (16, 6

In [76]:
# REUSABLE TABLE DISPLAY FUNCTION WITH CSV EXPORT
def display_workforce_table(table_number):
    """
    Display comprehensive information about a workforce table and export to CSV
    
    Parameters:
    table_number (int): The table number (e.g., 1, 2, 3, etc.)
    
    Usage:
    display_workforce_table(1)  # Shows workforce_table1 and exports to CSV
    display_workforce_table(2)  # Shows workforce_table2 and exports to CSV
    """
    
    # Table number to province/region mapping for CSV naming
    table_to_name = {
        1: 'canada_summary',
        2: 'newfoundland_labrador', 
        3: 'prince_edward_island',
        4: 'nova_scotia',
        5: 'new_brunswick', 
        6: 'quebec',
        7: 'ontario',
        8: 'manitoba',
        9: 'saskatchewan', 
        10: 'alberta',
        11: 'british_columbia',
        15: 'canada_by_profession',
        16: 'canada_demographics', 
        17: 'canada_trends'
    }
    
    # Construct the variable name
    var_name = f"workforce_table{table_number}"
    
    # Check if the table exists
    if var_name not in globals():
        print(f"ERROR: {var_name} not found!")
        print(f"Available workforce tables: {[v for v in globals() if v.startswith('workforce_table')]}")
        return
    
    # Get the table
    df = globals()[var_name]
    
    # Display header
    print("=" * 60)
    print(f"TABLE {table_number}")
    print("=" * 60)
    
    # Basic info
    print(f"Shape: {df.shape}")
    print(f"Columns ({len(df.columns)}): {list(df.columns)}")
    
    # Sample data
    print(f"\nFirst 10 rows:")
    display(df.head(10))
    
    # Data types
    print(f"\nData types:")
    print(df.dtypes)
    
    # Null values
    print(f"\nNull values count:")
    print(df.isnull().sum())
    
    # Additional info for tables with Year column
    if 'Year' in df.columns:
        unique_years = sorted(df['Year'].dropna().unique())
        print(f"\nYears in data: {unique_years}")
        print(f"Year column type: {df['Year'].dtype}")
    
    # CSV Export
    print(f"\n" + "="*40)
    print("CSV EXPORT:")
    
    try:
        # Create data directory if it doesn't exist
        import os
        if not os.path.exists('data'):
            os.makedirs('data')
            print("Created data/ directory")
        
        # Determine CSV filename
        if table_number in table_to_name:
            csv_name = f"workforce_{table_to_name[table_number]}.csv"
        else:
            csv_name = f"workforce_table{table_number}.csv"
        
        csv_path = f"data/{csv_name}"
        
        # Export to CSV
        df.to_csv(csv_path, index=False)
        print(f"SUCCESS: Exported to {csv_path}")
        print(f"File size: {os.path.getsize(csv_path)} bytes")
        
    except Exception as e:
        print(f"ERROR: CSV export failed - {str(e)}")
    
    print(f"\n" + "="*60)
    print(f"SUCCESS: {var_name} display and export complete!")

In [78]:
display_workforce_table(1) 

TABLE 1
Shape: (51, 29)
Columns (29): ['Type of professional', 'Newfoundland and Labrador: Count', 'Newfoundland and Labrador: Per 100,000 population', 'Prince Edward Island: Count', 'Prince Edward Island: Per 100,000 population', 'Nova Scotia: Count', 'Nova Scotia: Per 100,000 population', 'New Brunswick: Count', 'New Brunswick: Per 100,000 population', 'Quebec: Count', 'Quebec: Per 100,000 population', 'Ontario: Count', 'Ontario: Per 100,000 population', 'Manitoba: Count', 'Manitoba: Per 100,000 population', 'Saskatchewan: Count', 'Saskatchewan: Per 100,000 population', 'Alberta: Count', 'Alberta: Per 100,000 population', 'British Columbia: Count', 'British Columbia: Per 100,000 population', 'Yukon: Count', 'Yukon: Per 100,000 population', 'Northwest Territories: Count', 'Northwest Territories: Per 100,000 population', 'Nunavut: Count', 'Nunavut: Per 100,000 population', 'Provinces/territories with available data: Count', 'Provinces/territories with available data: Per 100,000 popula

,Type of professional,Newfoundland and Labrador: Count,"Newfoundland and Labrador: Per 100,000 population",Prince Edward Island: Count,"Prince Edward Island: Per 100,000 population",Nova Scotia: Count,"Nova Scotia: Per 100,000 population",New Brunswick: Count,"New Brunswick: Per 100,000 population",Quebec: Count,"Quebec: Per 100,000 population",Ontario: Count,"Ontario: Per 100,000 population",Manitoba: Count,"Manitoba: Per 100,000 population",Saskatchewan: Count,"Saskatchewan: Per 100,000 population",Alberta: Count,"Alberta: Per 100,000 population",British Columbia: Count,"British Columbia: Per 100,000 population",Yukon: Count,"Yukon: Per 100,000 population",Northwest Territories: Count,"Northwest Territories: Per 100,000 population",Nunavut: Count,"Nunavut: Per 100,000 population",Provinces/territories with available data: Count,"Provinces/territories with available data: Per 100,000 population"
0,Audiologists,50.0,9.3,9.0,5.2,112.0,10.6,85.0,10.2,545.0,6.1,888.0,5.7,67.0,4.6,45.0,3.70000,203.0,4.300000,309.0,5.600000,1.0,2.2,2.0,4.4,0.0,0.0,2316.0,5.800000
1,Chiropractors,NaN,NaN,15.0,8.6,NaN,NaN,118.0,14.1,1358.0,15.3,4996.0,32.0,286.0,19.7,244.0,20.20000,1272.0,27.100000,NaN,NaN,7.0,15.6,NaN,NaN,NaN,NaN,8296.0,25.200000
2,Combined laboratory and X-ray technologists,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,640.0,13.630681,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,640.0,13.630681
3,Dental assistants,358.0,66.5,NaN,NaN,874.0,82.6,615.0,73.7,NaN,NaN,5657.0,36.2,1312.0,90.2,1038.0,85.80000,6839.0,145.700000,6292.0,114.005892,NaN,NaN,NaN,NaN,NaN,NaN,22985.0,57.322403
4,Dental hygienists,302.0,56.1,131.0,75.4,836.0,79.0,576.0,69.0,7095.0,79.9,14498.0,92.9,884.0,60.8,720.0,59.54808,3921.0,83.500000,4550.0,82.442277,39.0,86.7,39.0,86.7,17.0,41.8,33608.0,83.815154
5,Dentists,215.0,39.9,81.0,46.6,569.0,53.7,379.0,45.4,5405.0,60.9,11983.0,76.8,809.0,55.6,576.0,47.60000,2894.0,61.600000,4114.0,74.542314,56.0,124.5,50.0,111.2,101.0,248.3,27232.0,67.914016
6,Denturists,21.0,3.9,3.0,1.7,29.0,2.7,31.0,3.7,859.0,9.7,777.0,5.0,43.0,3.0,70.0,5.80000,321.0,6.800000,253.0,4.600000,3.0,6.7,1.0,2.2,0.0,0.0,2411.0,6.000000
7,Dietitians,172.0,31.9,NaN,NaN,603.0,57.0,409.0,49.0,3458.0,39.0,4524.0,29.0,516.0,35.5,421.0,34.80000,1411.0,30.100000,1550.0,28.100000,NaN,NaN,NaN,NaN,NaN,NaN,13064.0,32.800000
8,Environmental public health professionals,17.0,3.2,106.0,8.6,106.0,8.6,46.0,0.5,46.0,0.5,674.0,4.3,70.0,4.8,100.0,8.30000,379.0,7.900000,244.0,4.400000,244.0,4.4,379.0,7.9,379.0,7.9,1636.0,4.100000
9,Genetic counsellors,4.0,0.7,1.0,0.6,16.0,1.5,0.0,0.0,59.0,0.7,144.0,0.9,11.0,0.8,7.0,0.60000,34.0,0.700000,49.0,0.900000,1.0,2.2,0.0,0.0,0.0,0.0,326.0,0.800000



Data types:
Type of professional                                                  object
Newfoundland and Labrador: Count                                     float64
Newfoundland and Labrador: Per 100,000 population                    float64
Prince Edward Island: Count                                          float64
Prince Edward Island: Per 100,000 population                         float64
Nova Scotia: Count                                                   float64
Nova Scotia: Per 100,000 population                                  float64
New Brunswick: Count                                                 float64
New Brunswick: Per 100,000 population                                float64
Quebec: Count                                                        float64
Quebec: Per 100,000 population                                       float64
Ontario: Count                                                       float64
Ontario: Per 100,000 population                                

In [82]:
display_workforce_table(2) 

TABLE 2
Shape: (195, 8)
Columns (8): ['Type of professional', 'Year', 'Newfoundland and Labrador: Count', 'Newfoundland and Labrador: Per 100,000 population', 'Newfoundland and Labrador: Female (%)', 'Newfoundland and Labrador: Age <30 years (%)', 'Newfoundland and Labrador: Age 30–59 years (%)', 'Newfoundland and Labrador: Age 60+ years (%)']

First 10 rows:


,Type of professional,Year,Newfoundland and Labrador: Count,"Newfoundland and Labrador: Per 100,000 population",Newfoundland and Labrador: Female (%),Newfoundland and Labrador: Age <30 years (%),Newfoundland and Labrador: Age 30–59 years (%),Newfoundland and Labrador: Age 60+ years (%)
0,Audiologists,2019,39.0,7.4,82.1,17.9,76.9,5.1
1,Audiologists,2020,43.0,8.2,83.7,23.3,69.8,7.0
2,Audiologists,2021,47.0,8.9,85.1,23.4,70.2,6.4
3,Audiologists,2022,48.0,9.0,85.4,20.8,72.9,6.2
4,Audiologists,2023,50.0,9.3,86.0,22.0,72.0,6.0
5,Chiropractors,2019,NaN,NaN,NaN,NaN,NaN,NaN
6,Chiropractors,2020,NaN,NaN,NaN,NaN,NaN,NaN
7,Chiropractors,2021,NaN,NaN,NaN,NaN,NaN,NaN
8,Chiropractors,2022,NaN,NaN,NaN,NaN,NaN,NaN
9,Chiropractors,2023,NaN,NaN,NaN,NaN,NaN,NaN



Data types:
Type of professional                                  object
Year                                                   Int64
Newfoundland and Labrador: Count                     float64
Newfoundland and Labrador: Per 100,000 population    float64
Newfoundland and Labrador: Female (%)                float64
Newfoundland and Labrador: Age <30 years (%)         float64
Newfoundland and Labrador: Age 30–59 years (%)       float64
Newfoundland and Labrador: Age 60+ years (%)         float64
dtype: object

Null values count:
Type of professional                                  0
Year                                                 10
Newfoundland and Labrador: Count                     51
Newfoundland and Labrador: Per 100,000 population    51
Newfoundland and Labrador: Female (%)                73
Newfoundland and Labrador: Age <30 years (%)         87
Newfoundland and Labrador: Age 30–59 years (%)       86
Newfoundland and Labrador: Age 60+ years (%)         86
dtype: int64

Yea

In [84]:

display_workforce_table(3)

TABLE 3
Shape: (196, 8)
Columns (8): ['Type of professional', 'Year', 'Prince Edward Island: Count', 'Prince Edward Island: Per 100,000 population', 'Prince Edward Island: Female (%)', 'Prince Edward Island: Age <30 years (%)', 'Prince Edward Island: Age 30–59 years (%)', 'Prince Edward Island: Age 60+ years (%)']

First 10 rows:


,Type of professional,Year,Prince Edward Island: Count,"Prince Edward Island: Per 100,000 population",Prince Edward Island: Female (%),Prince Edward Island: Age <30 years (%),Prince Edward Island: Age 30–59 years (%),Prince Edward Island: Age 60+ years (%)
0,Audiologists,2019,10.0,6.4,80.0,10.0,80.0,10.0
1,Audiologists,2020,10.0,6.3,80.0,10.0,80.0,10.0
2,Audiologists,2021,9.0,5.6,77.8,11.1,77.8,11.1
3,Audiologists,2022,9.0,5.4,77.8,11.1,77.8,11.1
4,Audiologists,2023,9.0,5.2,77.8,11.1,88.9,0.0
5,Chiropractors,2019,11.0,7.1,45.5,20.0,70.0,10.0
6,Chiropractors,2020,17.0,10.7,NaN,NaN,NaN,NaN
7,Chiropractors,2021,17.0,10.5,64.7,20.0,73.3,6.7
8,Chiropractors,2022,15.0,9.0,60.0,14.3,85.7,0.0
9,Chiropractors,2023,15.0,8.6,53.3,6.7,93.3,0.0



Data types:
Type of professional                             object
Year                                              Int64
Prince Edward Island: Count                     float64
Prince Edward Island: Per 100,000 population    float64
Prince Edward Island: Female (%)                float64
Prince Edward Island: Age <30 years (%)         float64
Prince Edward Island: Age 30–59 years (%)       float64
Prince Edward Island: Age 60+ years (%)         float64
dtype: object

Null values count:
Type of professional                             0
Year                                            11
Prince Edward Island: Count                     49
Prince Edward Island: Per 100,000 population    49
Prince Edward Island: Female (%)                84
Prince Edward Island: Age <30 years (%)         92
Prince Edward Island: Age 30–59 years (%)       90
Prince Edward Island: Age 60+ years (%)         92
dtype: int64

Years in data: [2019, 2020, 2021, 2022, 2023]
Year column type: Int64

CSV EXPORT:


In [86]:
display_workforce_table(4)

TABLE 4
Shape: (196, 8)
Columns (8): ['Type of professional', 'Year', 'Nova Scotia: Count', 'Nova Scotia: Per 100,000 population', 'Nova Scotia: Female (%)', 'Nova Scotia: Age <30 years (%)', 'Nova Scotia: Age 30–59 years (%)', 'Nova Scotia: Age 60+ years (%)']

First 10 rows:


,Type of professional,Year,Nova Scotia: Count,"Nova Scotia: Per 100,000 population",Nova Scotia: Female (%),Nova Scotia: Age <30 years (%),Nova Scotia: Age 30–59 years (%),Nova Scotia: Age 60+ years (%)
0,Audiologists,2019,101.0,10.4,75.2,6.9,78.2,14.9
1,Audiologists,2020,111.0,11.2,NaN,14.4,73.9,11.7
2,Audiologists,2021,115.0,11.5,NaN,13.9,76.5,9.6
3,Audiologists,2022,115.0,11.2,NaN,9.6,81.7,8.7
4,Audiologists,2023,112.0,10.6,NaN,11.6,78.6,9.8
5,Chiropractors,2019,173.0,17.7,45.1,7.3,86.0,6.7
6,Chiropractors,2020,179.0,18.1,49.7,8.0,86.2,5.7
7,Chiropractors,2021,178.0,17.8,53.0,10.3,84.0,5.7
8,Chiropractors,2022,NaN,NaN,NaN,NaN,NaN,NaN
9,Chiropractors,2023,NaN,NaN,NaN,NaN,NaN,NaN



Data types:
Type of professional                    object
Year                                     Int64
Nova Scotia: Count                     float64
Nova Scotia: Per 100,000 population    float64
Nova Scotia: Female (%)                float64
Nova Scotia: Age <30 years (%)         float64
Nova Scotia: Age 30–59 years (%)       float64
Nova Scotia: Age 60+ years (%)         float64
dtype: object

Null values count:
Type of professional                    0
Year                                   11
Nova Scotia: Count                     45
Nova Scotia: Per 100,000 population    45
Nova Scotia: Female (%)                74
Nova Scotia: Age <30 years (%)         79
Nova Scotia: Age 30–59 years (%)       79
Nova Scotia: Age 60+ years (%)         79
dtype: int64

Years in data: [2019, 2020, 2021, 2022, 2023]
Year column type: Int64

CSV EXPORT:
SUCCESS: Exported to data/workforce_nova_scotia.csv
File size: 11123 bytes

SUCCESS: workforce_table4 display and export complete!


In [88]:
display_workforce_table(5)

TABLE 5
Shape: (196, 8)
Columns (8): ['Type of professional', 'Year', 'New Brunswick: Count', 'New Brunswick: Per 100,000 population', 'New Brunswick: Female (%)', 'New Brunswick: Age <30 years (%)', 'New Brunswick: Age 30–59 years (%)', 'New Brunswick: Age 60+ years (%)']

First 10 rows:


,Type of professional,Year,New Brunswick: Count,"New Brunswick: Per 100,000 population",New Brunswick: Female (%),New Brunswick: Age <30 years (%),New Brunswick: Age 30–59 years (%),New Brunswick: Age 60+ years (%)
0,Audiologists,2019,80.0,10.300000,72.5,6.3,83.8,10.0
1,Audiologists,2020,82.0,10.500000,75.6,9.8,80.5,9.8
2,Audiologists,2021,82.0,10.400000,80.5,9.8,84.1,6.1
3,Audiologists,2022,79.0,9.800000,83.5,11.4,86.1,2.5
4,Audiologists,2023,85.0,10.200000,84.7,10.6,85.9,3.5
5,Chiropractors,2019,99.0,12.700000,43.4,21.2,66.7,12.1
6,Chiropractors,2020,110.0,14.040785,46.4,24.5,61.8,13.6
7,Chiropractors,2021,109.0,13.800000,45.0,24.8,60.6,14.7
8,Chiropractors,2022,113.0,14.000000,49.6,26.5,59.3,14.2
9,Chiropractors,2023,118.0,14.100000,44.9,NaN,NaN,NaN



Data types:
Type of professional                      object
Year                                       Int64
New Brunswick: Count                     float64
New Brunswick: Per 100,000 population    float64
New Brunswick: Female (%)                float64
New Brunswick: Age <30 years (%)         float64
New Brunswick: Age 30–59 years (%)       float64
New Brunswick: Age 60+ years (%)         float64
dtype: object

Null values count:
Type of professional                      0
Year                                     11
New Brunswick: Count                     41
New Brunswick: Per 100,000 population    41
New Brunswick: Female (%)                66
New Brunswick: Age <30 years (%)         77
New Brunswick: Age 30–59 years (%)       76
New Brunswick: Age 60+ years (%)         76
dtype: int64

Years in data: [2019, 2020, 2021, 2022, 2023]
Year column type: Int64

CSV EXPORT:
SUCCESS: Exported to data/workforce_new_brunswick.csv
File size: 11201 bytes

SUCCESS: workforce_table5 display 

In [90]:
display_workforce_table(6)

TABLE 6
Shape: (197, 8)
Columns (8): ['Type of professional', 'Year', 'Quebec: Count', 'Quebec: Per 100,000 population', 'Quebec: Female (%)', 'Quebec: Age <30 years (%)', 'Quebec: Age 30–59 years (%)', 'Quebec: Age 60+ years (%)']

First 10 rows:


,Type of professional,Year,Quebec: Count,"Quebec: Per 100,000 population",Quebec: Female (%),Quebec: Age <30 years (%),Quebec: Age 30–59 years (%),Quebec: Age 60+ years (%)
0,Audiologists,2019,471.0,5.6,87.7,NaN,NaN,NaN
1,Audiologists,2020,482.0,5.6,88.4,16.2,76.8,7.1
2,Audiologists,2021,481.0,5.6,88.1,16.0,77.5,6.4
3,Audiologists,2022,499.0,5.8,87.8,18.0,76.4,5.6
4,Audiologists,2023,545.0,6.1,87.3,21.7,73.0,5.3
5,Chiropractors,2019,1369.0,16.1,48.3,12.0,71.8,16.2
6,Chiropractors,2020,NaN,NaN,NaN,NaN,NaN,NaN
7,Chiropractors,2021,1330.0,15.5,50.1,9.5,72.1,18.3
8,Chiropractors,2022,1352.0,15.6,51.0,12.2,71.6,16.2
9,Chiropractors,2023,1358.0,15.3,52.0,14.7,70.4,14.9



Data types:
Type of professional               object
Year                                Int64
Quebec: Count                     float64
Quebec: Per 100,000 population    float64
Quebec: Female (%)                float64
Quebec: Age <30 years (%)         float64
Quebec: Age 30–59 years (%)       float64
Quebec: Age 60+ years (%)         float64
dtype: object

Null values count:
Type of professional               0
Year                              12
Quebec: Count                     49
Quebec: Per 100,000 population    49
Quebec: Female (%)                68
Quebec: Age <30 years (%)         82
Quebec: Age 30–59 years (%)       82
Quebec: Age 60+ years (%)         82
dtype: int64

Years in data: [2019, 2020, 2021, 2022, 2023]
Year column type: Int64

CSV EXPORT:
SUCCESS: Exported to data/workforce_quebec.csv
File size: 11549 bytes

SUCCESS: workforce_table6 display and export complete!


In [92]:
display_workforce_table(7)

TABLE 7
Shape: (195, 8)
Columns (8): ['Type of professional', 'Year', 'Ontario: Count', 'Ontario: Per 100,000 population', 'Ontario: Female (%)', 'Ontario: Age <30 years (%)', 'Ontario: Age 30–59 years (%)', 'Ontario: Age 60+ years (%)']

First 10 rows:


,Type of professional,Year,Ontario: Count,"Ontario: Per 100,000 population",Ontario: Female (%),Ontario: Age <30 years (%),Ontario: Age 30–59 years (%),Ontario: Age 60+ years (%)
0,Audiologists,2019,789.0,5.4,80.0,15.3,75.3,9.4
1,Audiologists,2020,793.0,5.4,79.7,16.1,75.2,8.7
2,Audiologists,2021,840.0,5.7,80.1,14.6,75.6,9.8
3,Audiologists,2022,867.0,5.7,80.6,14.9,75.9,9.2
4,Audiologists,2023,888.0,5.7,80.6,14.6,75.7,9.7
5,Chiropractors,2019,4626.0,31.7,39.4,7.2,79.3,13.5
6,Chiropractors,2020,4699.0,31.8,40.0,7.4,79.2,13.4
7,Chiropractors,2021,4792.0,32.3,40.8,11.1,76.9,12.0
8,Chiropractors,2022,4878.0,32.2,41.0,11.2,77.2,11.6
9,Chiropractors,2023,4996.0,32.0,41.3,11.1,76.6,12.3



Data types:
Type of professional                object
Year                                 Int64
Ontario: Count                     float64
Ontario: Per 100,000 population    float64
Ontario: Female (%)                float64
Ontario: Age <30 years (%)         float64
Ontario: Age 30–59 years (%)       float64
Ontario: Age 60+ years (%)         float64
dtype: object

Null values count:
Type of professional                0
Year                               10
Ontario: Count                     34
Ontario: Per 100,000 population    34
Ontario: Female (%)                56
Ontario: Age <30 years (%)         64
Ontario: Age 30–59 years (%)       64
Ontario: Age 60+ years (%)         64
dtype: int64

Years in data: [2019, 2020, 2021, 2022, 2023]
Year column type: Int64

CSV EXPORT:
SUCCESS: Exported to data/workforce_ontario.csv
File size: 11241 bytes

SUCCESS: workforce_table7 display and export complete!


In [94]:
display_workforce_table(8)

TABLE 8
Shape: (195, 8)
Columns (8): ['Type of professional', 'Year', 'Manitoba: Count', 'Manitoba: Per 100,000 population', 'Manitoba: Female (%)', 'Manitoba: Age <30 years (%)', 'Manitoba: Age 30–59 years (%)', 'Manitoba: Age 60+ years (%)']

First 10 rows:


,Type of professional,Year,Manitoba: Count,"Manitoba: Per 100,000 population",Manitoba: Female (%),Manitoba: Age <30 years (%),Manitoba: Age 30–59 years (%),Manitoba: Age 60+ years (%)
0,Audiologists,2019,68.0,5.0,NaN,NaN,NaN,NaN
1,Audiologists,2020,69.0,5.0,NaN,NaN,NaN,NaN
2,Audiologists,2021,77.0,5.5,NaN,NaN,NaN,NaN
3,Audiologists,2022,77.0,5.4,NaN,NaN,NaN,NaN
4,Audiologists,2023,67.0,4.6,NaN,NaN,NaN,NaN
5,Chiropractors,2019,294.0,21.5,29.6,5.7,78.9,15.4
6,Chiropractors,2020,296.0,21.4,30.7,6.4,78.4,15.2
7,Chiropractors,2021,294.0,21.1,31.3,6.1,78.2,15.6
8,Chiropractors,2022,289.0,20.4,30.8,5.5,75.8,18.7
9,Chiropractors,2023,286.0,19.7,32.5,4.2,75.9,19.9



Data types:
Type of professional                 object
Year                                  Int64
Manitoba: Count                     float64
Manitoba: Per 100,000 population    float64
Manitoba: Female (%)                float64
Manitoba: Age <30 years (%)         float64
Manitoba: Age 30–59 years (%)       float64
Manitoba: Age 60+ years (%)         float64
dtype: object

Null values count:
Type of professional                 0
Year                                10
Manitoba: Count                     34
Manitoba: Per 100,000 population    34
Manitoba: Female (%)                83
Manitoba: Age <30 years (%)         85
Manitoba: Age 30–59 years (%)       85
Manitoba: Age 60+ years (%)         85
dtype: int64

Years in data: [2019, 2020, 2021, 2022, 2023]
Year column type: Int64

CSV EXPORT:
SUCCESS: Exported to data/workforce_manitoba.csv
File size: 10600 bytes

SUCCESS: workforce_table8 display and export complete!


In [96]:
display_workforce_table(9)

TABLE 9
Shape: (195, 8)
Columns (8): ['Type of professional', 'Year', 'Saskatchewan: Count', 'Saskatchewan: Per 100,000 population', 'Saskatchewan: Female (%)', 'Saskatchewan: Age <30 years (%)', 'Saskatchewan: Age 30–59 years (%)', 'Saskatchewan: Age 60+ years (%)']

First 10 rows:


,Type of professional,Year,Saskatchewan: Count,"Saskatchewan: Per 100,000 population",Saskatchewan: Female (%),Saskatchewan: Age <30 years (%),Saskatchewan: Age 30–59 years (%),Saskatchewan: Age 60+ years (%)
0,Audiologists,2019,NaN,NaN,NaN,NaN,NaN,NaN
1,Audiologists,2020,46.0,3.9,93.0,7.0,86.0,7.0
2,Audiologists,2021,46.0,3.9,89.1,4.3,91.3,4.3
3,Audiologists,2022,46.0,3.9,91.3,6.5,84.8,8.7
4,Audiologists,2023,45.0,3.7,86.7,8.9,84.4,6.7
5,Chiropractors,2019,218.0,18.7,35.8,6.4,74.3,19.3
6,Chiropractors,2020,223.0,19.1,35.4,10.8,71.7,17.5
7,Chiropractors,2021,227.0,19.4,39.2,11.9,72.7,15.4
8,Chiropractors,2022,238.0,20.2,39.1,12.2,72.3,15.5
9,Chiropractors,2023,244.0,20.2,40.2,10.7,75.4,13.9



Data types:
Type of professional                     object
Year                                      Int64
Saskatchewan: Count                     float64
Saskatchewan: Per 100,000 population    float64
Saskatchewan: Female (%)                float64
Saskatchewan: Age <30 years (%)         float64
Saskatchewan: Age 30–59 years (%)       float64
Saskatchewan: Age 60+ years (%)         float64
dtype: object

Null values count:
Type of professional                     0
Year                                    10
Saskatchewan: Count                     34
Saskatchewan: Per 100,000 population    34
Saskatchewan: Female (%)                68
Saskatchewan: Age <30 years (%)         72
Saskatchewan: Age 30–59 years (%)       72
Saskatchewan: Age 60+ years (%)         72
dtype: int64

Years in data: [2019, 2020, 2021, 2022, 2023]
Year column type: Int64

CSV EXPORT:
SUCCESS: Exported to data/workforce_saskatchewan.csv
File size: 11203 bytes

SUCCESS: workforce_table9 display and export comple

In [98]:
display_workforce_table(10)

TABLE 10
Shape: (196, 8)
Columns (8): ['Type of professional', 'Year', 'Alberta: Count', 'Alberta: Per 100,000 population', 'Alberta: Female (%)', 'Alberta: Age <30 years (%)', 'Alberta: Age 30–59 years (%)', 'Alberta: Age 60+ years (%)']

First 10 rows:


,Type of professional,Year,Alberta: Count,"Alberta: Per 100,000 population",Alberta: Female (%),Alberta: Age <30 years (%),Alberta: Age 30–59 years (%),Alberta: Age 60+ years (%)
0,Audiologists,2019,186.0,4.3,78.5,11.3,80.1,8.6
1,Audiologists,2020,197.0,4.5,79.7,10.2,80.7,9.1
2,Audiologists,2021,205.0,4.6,79.0,11.7,77.1,11.2
3,Audiologists,2022,202.0,4.5,78.7,10.4,78.7,10.9
4,Audiologists,2023,203.0,4.3,78.8,7.9,79.8,12.3
5,Chiropractors,2019,1178.0,27.0,31.7,9.2,80.0,10.9
6,Chiropractors,2020,1159.0,26.3,31.0,10.0,79.4,10.6
7,Chiropractors,2021,1174.0,26.5,31.8,8.3,80.4,11.3
8,Chiropractors,2022,1238.0,27.4,32.6,10.5,77.7,11.8
9,Chiropractors,2023,1272.0,27.1,33.3,11.8,77.4,10.8



Data types:
Type of professional                object
Year                                 Int64
Alberta: Count                     float64
Alberta: Per 100,000 population    float64
Alberta: Female (%)                float64
Alberta: Age <30 years (%)         float64
Alberta: Age 30–59 years (%)       float64
Alberta: Age 60+ years (%)         float64
dtype: object

Null values count:
Type of professional                0
Year                               11
Alberta: Count                     24
Alberta: Per 100,000 population    24
Alberta: Female (%)                47
Alberta: Age <30 years (%)         67
Alberta: Age 30–59 years (%)       67
Alberta: Age 60+ years (%)         67
dtype: int64

Years in data: [2019, 2020, 2021, 2022, 2023]
Year column type: Int64

CSV EXPORT:
SUCCESS: Exported to data/workforce_alberta.csv
File size: 11755 bytes

SUCCESS: workforce_table10 display and export complete!


In [100]:
display_workforce_table(11)

TABLE 11
Shape: (196, 8)
Columns (8): ['Type of professional', 'Year', 'British Columbia: Count', 'British Columbia: Per 100,000 population', 'British Columbia: Female (%)', 'British Columbia: Age <30 years (%)', 'British Columbia: Age 30–59 years (%)', 'British Columbia: Age 60+ years (%)']

First 10 rows:


,Type of professional,Year,British Columbia: Count,"British Columbia: Per 100,000 population",British Columbia: Female (%),British Columbia: Age <30 years (%),British Columbia: Age 30–59 years (%),British Columbia: Age 60+ years (%)
0,Audiologists,2019,310.0,6.1,79.0,8.7,78.4,12.9
1,Audiologists,2020,314.0,6.1,79.0,8.0,79.3,12.7
2,Audiologists,2021,315.0,6.0,81.0,10.8,76.8,12.4
3,Audiologists,2022,306.0,5.7,80.7,5.9,79.1,15.0
4,Audiologists,2023,309.0,5.6,80.3,7.1,80.6,12.3
5,Chiropractors,2019,1324.0,25.9,32.6,12.5,73.3,14.2
6,Chiropractors,2020,1358.0,26.2,33.9,9.4,76.3,14.3
7,Chiropractors,2021,1394.0,26.7,34.1,9.8,75.6,14.6
8,Chiropractors,2022,NaN,NaN,NaN,NaN,NaN,NaN
9,Chiropractors,2023,NaN,NaN,NaN,NaN,NaN,NaN



Data types:
Type of professional                         object
Year                                          Int64
British Columbia: Count                     float64
British Columbia: Per 100,000 population    float64
British Columbia: Female (%)                float64
British Columbia: Age <30 years (%)         float64
British Columbia: Age 30–59 years (%)       float64
British Columbia: Age 60+ years (%)         float64
dtype: object

Null values count:
Type of professional                         0
Year                                        11
British Columbia: Count                     40
British Columbia: Per 100,000 population    40
British Columbia: Female (%)                75
British Columbia: Age <30 years (%)         93
British Columbia: Age 30–59 years (%)       93
British Columbia: Age 60+ years (%)         93
dtype: int64

Years in data: [2019, 2020, 2021, 2022, 2023]
Year column type: Int64

CSV EXPORT:
SUCCESS: Exported to data/workforce_british_columbia.csv
File si

In [102]:
display_workforce_table(15)

TABLE 15
Shape: (16, 6)
Columns (6): ['Province/territory', '2019', '2020', '2021', '2022', '2023']

First 10 rows:


,Province/territory,2019,2020,2021,2022,2023
0,Newfoundland and Labrador,527643.0,526884.0,527056.0,531583.0,538605.0
1,Prince Edward Island,155792.0,159193.0,162133.0,167188.0,173787.0
2,Nova Scotia,975799.0,989168.0,999908.0,1025445.0,1058694.0
3,New Brunswick,777387.0,783432.0,790802.0,809568.0,834691.0
4,Quebec,8483186.0,8551095.0,8572020.0,8672185.0,8874683.0
5,Ontario,14573565.0,14761811.0,14842488.0,15145006.0,15608369.0
6,Manitoba,1370033.0,1380132.0,1391924.0,1413409.0,1454902.0
7,Saskatchewan,1164223.0,1167386.0,1167711.0,1178422.0,1209107.0
8,Alberta,4355377.0,4407495.0,4431531.0,4510891.0,4695290.0
9,British Columbia,5111022.0,5176101.0,5226665.0,5356284.0,5519013.0



Data types:
Province/territory     object
2019                  float64
2020                  float64
2021                  float64
2022                  float64
2023                  float64
dtype: object

Null values count:
Province/territory    0
2019                  3
2020                  3
2021                  3
2022                  3
2023                  3
dtype: int64

CSV EXPORT:
SUCCESS: Exported to data/workforce_canada_by_profession.csv
File size: 936 bytes

SUCCESS: workforce_table15 display and export complete!


In [104]:
display_workforce_table(16)

TABLE 16
Shape: (81, 14)
Columns (14): ['Type of professional', 'N.L.', 'P.E.I.', 'N.S.', 'N.B.', 'Que.', 'Ont.', 'Man. ', 'Sask.', 'Alta.', 'B.C.', 'Y.T.', 'N.W.T. ', 'Nun.']

First 10 rows:


,Type of professional,N.L.,P.E.I.,N.S.,N.B.,Que.,Ont.,Man.,Sask.,Alta.,B.C.,Y.T.,N.W.T.,Nun.
0,Audiologists,N,N,Y,N,Y,Y,N,N,Y,Y,N,N,N
1,Chiropractors,N,N,N,N,Y,Y,N,N,N,N,N,N,N
2,Combined laboratory and X-ray technologists,N,N,N,N,N,N,N,Y,Y,N,N,N,N
3,Dental assistants,Y,Y,Y,Y,N,Y,Y,Y,Y,Y,N,N,N
4,Dental hygienists,N,N,Y,Y,Y,Y,Y,Y,Y,Y,N,N,N
5,Dentists,N,N,Y,N,Y,Y,Y,Y,Y,Y,N,N,N
6,Denturists,N,N,N,N,Y,Y,N,N,Y,N,N,N,N
7,Dietitians,N,Y,Y,Y,Y,Y,Y,Y,Y,Y,N,N,N
8,Environmental public health professionals,N,N,Y,N,Y,Y,N,Y,Y,Y,N,N,N
9,Genetic counsellors,N,N,N,N,Y,Y,Y,N,N,Y,N,N,N



Data types:
Type of professional    object
N.L.                    object
P.E.I.                  object
N.S.                    object
N.B.                    object
Que.                    object
Ont.                    object
Man.                    object
Sask.                   object
Alta.                   object
B.C.                    object
Y.T.                    object
N.W.T.                  object
Nun.                    object
dtype: object

Null values count:
Type of professional     0
N.L.                    18
P.E.I.                  18
N.S.                    18
N.B.                    18
Que.                    18
Ont.                    18
Man.                    17
Sask.                   17
Alta.                   17
B.C.                    17
Y.T.                    17
N.W.T.                  18
Nun.                    18
dtype: int64

CSV EXPORT:
SUCCESS: Exported to data/workforce_canada_demographics.csv
File size: 5002 bytes

SUCCESS: workforce_table16 displ